In [1]:
import sys
sys.path.append('../')
from utils import preprocess

import numpy as np
import h5py

import pandas as pd

from collections import defaultdict
import string

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

import keras
from keras.layers import Dense, GlobalAveragePooling1D, Embedding, Lambda
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

np.random.seed(1234)

Using TensorFlow backend.


In [2]:
def create_docs(df, n_gram_max=1):
    docs = []

    for i, text in enumerate(df.text):    
        def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams

        doc = preprocess(text).split()
                        
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
        
    return docs

In [3]:
df = pd.read_csv('./../data/train_feature.csv')
df_test = pd.read_csv('./../data/test_feature.csv')
text = df.text.values
text_test = df_test.text.values

author2class = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
class2author = ['EAP', 'HPL', 'MWS']
y = np.array([author2class[a] for a in df.author])
y = to_categorical(y)

In [4]:
def create_model(input_dim, embeddings_dims=10):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [5]:
n_gram_max = 2
embedding_dims = 10

raw_docs = create_docs(df, n_gram_max=n_gram_max)
raw_docs_test = create_docs(df_test, n_gram_max=n_gram_max)

seed = 7
num_split = 5
epochs = 50

# for next training
predict_prob_features = np.zeros((len(df), 3))
predict_prob_features_test = np.zeros((len(df_test), 3))

ite = 0
sum_loss = 0.
min_count = 2

kf = KFold(n_splits=num_split, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(text):
    ite += 1
    print('{}/{}: #Trains: {}, #Val: {}'.format(ite, num_split, len(train_index), len(val_index)), end=' ')
    
    docs_train = [raw_docs[i] for i in train_index]
    docs_val = [raw_docs[i] for i in val_index]

    # get vocab
    tokenizer = Tokenizer(filters='', lower=False)
    #     tokenizer.fit_on_texts(docs_train)
    tokenizer.fit_on_texts(docs_train + docs_val)

    num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])
    
    tokenizer = Tokenizer(num_words=num_words, filters='', lower=False)
    tokenizer.fit_on_texts(docs_train + docs_val)    

    docs_train = tokenizer.texts_to_sequences(docs_train)
    docs_val   = tokenizer.texts_to_sequences(docs_val)
    docs_test  = tokenizer.texts_to_sequences(raw_docs_test)

    maxlen = 256
    x_train = pad_sequences(sequences=docs_train, maxlen=maxlen)
    x_val  = pad_sequences(sequences=docs_val, maxlen=maxlen)
    x_test = pad_sequences(sequences=docs_test, maxlen=maxlen)

    y_train, y_val = y[train_index], y[val_index]

    input_dim = max(np.max(x_train), np.max(x_val)) + 1
    print('#vocab: {} '.format(num_words), end=' ')
    print(x_train.shape, x_val.shape, x_test.shape)

    model = create_model(input_dim)
    
    checkpointer = ModelCheckpoint(filepath='./../fasttext_weights/weights.hdf5', verbose=0, save_best_only=True)

    hist = model.fit(x_train, y_train,
                     batch_size=16,
                     validation_data=(x_val, y_val),
                     epochs=epochs,
                     callbacks=[EarlyStopping(patience=4, monitor='val_loss'), 
                                checkpointer])

    # load best weights
    model.load_weights('./../fasttext_weights/weights.hdf5')
    y_pred = model.predict_proba(x_val)
    sum_loss += log_loss(y_pred=y_pred, y_true=np.nonzero(y_val)[1])
    
    # save features
    predict_prob_features[val_index] = y_pred
    predict_prob_features_test += model.predict_proba(x_test)
    print('valLoss: {}'.format(sum_loss/ite))

1/5: #Trains: 15663, #Val: 3916 #vocab: 76629  (15663, 256) (3916, 256) (8392, 256)
Train on 15663 samples, validate on 3916 samples
Epoch 1/50
15663/15663 [==============================] - 16s - loss: 1.0761 - acc: 0.4045 - val_loss: 1.0558 - val_acc: 0.4076
Epoch 2/50
15663/15663 [==============================] - 17s - loss: 1.0001 - acc: 0.5006 - val_loss: 0.9562 - val_acc: 0.5409
Epoch 3/50
15663/15663 [==============================] - 17s - loss: 0.8566 - acc: 0.7141 - val_loss: 0.8264 - val_acc: 0.6941
Epoch 4/50
15663/15663 [==============================] - 17s - loss: 0.7121 - acc: 0.7987 - val_loss: 0.7217 - val_acc: 0.7449
Epoch 5/50
15663/15663 [==============================] - 18s - loss: 0.5964 - acc: 0.8406 - val_loss: 0.6467 - val_acc: 0.7579
Epoch 6/50
15663/15663 [==============================] - 18s - loss: 0.5053 - acc: 0.8691 - val_loss: 0.5855 - val_acc: 0.7863
Epoch 7/50
15663/15663 [==============================] - 18s - loss: 0.4311 - acc: 0.8889 - val_lo

15663/15663 [==============================] - 16s - loss: 0.4250 - acc: 0.8909 - val_loss: 0.5261 - val_acc: 0.8095
Epoch 8/50
15663/15663 [==============================] - 16s - loss: 0.3626 - acc: 0.9110 - val_loss: 0.4834 - val_acc: 0.8299
Epoch 9/50
15663/15663 [==============================] - 16s - loss: 0.3101 - acc: 0.9259 - val_loss: 0.4535 - val_acc: 0.8394
Epoch 10/50
15663/15663 [==============================] - 17s - loss: 0.2654 - acc: 0.9396 - val_loss: 0.4291 - val_acc: 0.8460
Epoch 11/50
15663/15663 [==============================] - 16s - loss: 0.2276 - acc: 0.9499 - val_loss: 0.4065 - val_acc: 0.8519
Epoch 12/50
15663/15663 [==============================] - 16s - loss: 0.1955 - acc: 0.9586 - val_loss: 0.3993 - val_acc: 0.8473
Epoch 13/50
15663/15663 [==============================] - 17s - loss: 0.1679 - acc: 0.9655 - val_loss: 0.3762 - val_acc: 0.8539
Epoch 14/50
15663/15663 [==============================] - 17s - loss: 0.1448 - acc: 0.9710 - val_loss: 0.3664 

15664/15664 [==============================] - 16s - loss: 0.1128 - acc: 0.9801 - val_loss: 0.3590 - val_acc: 0.8600
Epoch 17/50
15664/15664 [==============================] - 15s - loss: 0.0976 - acc: 0.9831 - val_loss: 0.3538 - val_acc: 0.8603
Epoch 18/50
15664/15664 [==============================] - 16s - loss: 0.0842 - acc: 0.9859 - val_loss: 0.3537 - val_acc: 0.8593
Epoch 19/50
15664/15664 [==============================] - 15s - loss: 0.0728 - acc: 0.9883 - val_loss: 0.3606 - val_acc: 0.8580
Epoch 20/50
15664/15664 [==============================] - 14s - loss: 0.0633 - acc: 0.9898 - val_loss: 0.3462 - val_acc: 0.8636
Epoch 21/50
15664/15664 [==============================] - 18s - loss: 0.0550 - acc: 0.9912 - val_loss: 0.3483 - val_acc: 0.8656
Epoch 22/50
15664/15664 [==============================] - 16s - loss: 0.0480 - acc: 0.9927 - val_loss: 0.3461 - val_acc: 0.8692
Epoch 23/50
15664/15664 [==============================] - 17s - loss: 0.0416 - acc: 0.9936 - val_loss: 0.347

In [6]:
for a, c in author2class.items():
    df['{}_fasttext_bigram'.format(a)] = predict_prob_features[:, c]
    df_test['{}_fasttext_bigram'.format(a)] = predict_prob_features_test[:, c]/num_split

In [11]:
df.to_csv('./../data/train_feature.csv')
df_test.to_csv('./../data/test_feature.csv')